In [3]:
from evaluations import *

acc =1e-2
boundary = 'robin'
coarse_grid_resolution = 10

k=6.
cglob = -1j*k
cloc0 = 0
cloc1 = 0.02*(5-1j)
cloc2 = 0.0016*(8-1j)

p = helmholtz(boundary = boundary)
cloc = cloc0+ cloc1*k+cloc2*k**2
mus = {'k': k, 'c_glob': cglob, 'c_loc': cloc}
resolution  = int(np.ceil(float(k*1.5+50)/coarse_grid_resolution)*coarse_grid_resolution)
gq, lq = localize_problem(p, coarse_grid_resolution, resolution, mus = mus, calT = True, calQ = True)

calculate_continuity_constant(gq, lq)
calculate_inf_sup_constant2(gq, lq)
calculate_lambda_min(gq, lq)
calculate_csis(gq,lq)
calculate_Psi_norm(gq,lq)

d = gq["d"]
u = d.solve(mus)
bases = create_bases(gq, lq, num_testvecs=20, transfer = 'robin', target_accuracy = acc, calC = False)
ru = reconstruct_solution(gq,lq,bases)

norm = gq["full_norm"]
ls = norm(u-ru)/norm(u)
localizer = gq["localizer"]
pou = gq["pou"]

localizing problem
calculated_continuity_constant:  1.79186818672
calculated_inf_sup_constant:  0.227325649277
calculated all lambdas
calculated csis
calculated all Psi_norms


In [11]:
sum1 = NumpyVectorArray(u.data*0)
sum2 = 0
sum3 = 0
sum4 = 0
sum51 = 0
sum52 = 0
sum6 = 0
for space in gq["spaces"]:
    ldict = lq[space]
    basis = bases[space]
    M = ldict["range_product"]._matrix
    S = ldict["source_product"]._matrix
    M_sparse = scipy.sparse.csr_matrix(M)
    T = ldict["transfer_matrix_robin"]
    B = basis._array.T
    range_space = ldict["range_space"]
    omega_star_space = ldict["omega_star_space"]
    u_s = ldict["local_solution_robin"]
    u_s2 = ldict["local_sol2"]
    local_norm1 = induced_norm(ldict["range_product"])
    local_norm2 = induced_norm(ldict["omega_star_product"])

    u_loc = pou[range_space](localizer.localize_vector_array(u, range_space))
    u_loc2 = localizer.localize_vector_array(u, omega_star_space)
    u_dif = u_loc-u_s
    u_dif2 = u_loc2-u_s2
    term1 = u_dif.data.T - B.dot(B.conj().T).dot(M_sparse.dot(u_dif.data.T))
    term2 = NumpyVectorArray(term1.T)
    
    T1 = T - B.dot(B.conj().T).dot(M_sparse.dot(T))
    maxval = operator_svd2(T1, S, M_sparse)[0][0]
    
    sum1 += localizer.globalize_vector_array(term2, range_space)
    sum2 += local_norm1(term2)
    sum3 += maxval*ldict["Psi_norm"]*local_norm2(u_dif2)
    sum4 += maxval*ldict["Psi_norm"]*local_norm2(u_loc2)*ldict["csi"]
    sum51 += maxval**2*ldict["Psi_norm"]**2*ldict["csi"]**2
    sum52 += local_norm2(u_loc2)**2
rs1 = gq["continuity_constant"]/gq["inf_sup_constant"]* norm(sum1)/norm(u)
rs2 = gq["continuity_constant"]/gq["inf_sup_constant"]* sum2/norm(u)
rs3 = gq["continuity_constant"]/gq["inf_sup_constant"]* sum3/norm(u)
rs4 = gq["continuity_constant"]/gq["inf_sup_constant"]* sum4/norm(u)
rs5 = gq["continuity_constant"]/gq["inf_sup_constant"]* np.sqrt(sum51) * np.sqrt(sum52)/norm(u)
rs6 = gq["continuity_constant"]/gq["inf_sup_constant"]* np.sqrt(sum51) *4
ccs = gq["continuity_constant"]/gq["inf_sup_constant"]

In [13]:
print rs1/ls
print rs2/rs1
print rs3/rs2
print rs4/rs3
print rs5/rs4
print rs6/rs5

[ 11.25464672]
[ 7.43895931]
[ 3.70649843]
[ 1.28244783]
[ 1.10396197]
[ 1.20409607]
